In [1]:
import torch 
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MINST_data', download = True, train = True, transform= transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)
# Download and load the testing data
testset = datasets.FashionMNIST('~/.pytorch/F_MINST_data', download = True, train = False, transform= transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = True)


In [8]:
from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64,10)
        
        #define dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        #make sure the input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [10]:
model = Classifier()

images, labels = next(iter(testloader))
ps = torch.exp(model(images)) # get class probabilities
print(ps.shape)

torch.Size([64, 10])


In [11]:
top_p, top_class = ps.topk(1, dim = 1)
print(top_class[:10,:])

tensor([[4],
        [0],
        [4],
        [4],
        [0],
        [0],
        [4],
        [4],
        [0],
        [4]])


In [12]:
equals = top_class == labels.view(*top_class.shape)

In [13]:
equals


tensor([[0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]], dtype=torch.uint8)

In [20]:
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')

Accuracy: 7.8125%


In [ ]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.003)

epochs =30
steps =0
train_losses, test_losses = [] ,[]
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        
        optimizer.zero_grad()
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    else:
        
        test_loss = 0
        accuracy = 0
        
        #Turn off gradients for validation, saves memory and computation
        with torch.no_grad():
            model.eval() # to remove the dropout for validation
            for images, labels in testloader:
                log_ps = model(images)
                test_loss +=criterion(log_ps, labels)
            
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim = 1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        model.train() # set the model to train mode
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))
       
        print("Epoch: {}/{}..".format(e+1,epochs),
              "Training Loss: {:.3f}..".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

Epoch: 1/30.. Training Loss: 0.603.. Test Loss: 0.491 Test Accuracy: 0.827
Epoch: 2/30.. Training Loss: 0.483.. Test Loss: 0.448 Test Accuracy: 0.839
Epoch: 3/30.. Training Loss: 0.451.. Test Loss: 0.432 Test Accuracy: 0.842


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_losses, label = 'Training loss')
plt.plot(test_losses, label = 'Validation loss')
plt.legend(frameon=False)

In [ ]:
#import helper module
import helper

model.eval()
dataiter = iter(testloader)
images, labels = dataiter.next()
img = images[0]

img = img.view(1,784)

#Calculate the class probabilities (softmax) for img
with torch.no_grad():
    output= model.forward(img)
    
ps = torch.exp(output)

#Plot the image and probabilities
helper.view_classify(img.view(1,28,28), ps, version= "Fashion")